In [1]:
import endomill
import yaml


In [2]:
parameter_packs = [
    {
        "MILL_FITNESS_ADVANTAGE": fitness_advantage,
        "MILL_REPLICATE": replicate,
    }
    for replicate in range(10)
    for fitness_advantage in [0.0, 0.1, 1.0]
]

for i, pp in enumerate(parameter_packs):
    with open(
        f"gene-selection-inference-parameterization-{i}.yaml", "w"
    ) as file:
        yaml.safe_dump(pp, file)
# endomill.instantiate_over(parameter_packs=parameter_packs)


# Papermill Parameters


In [3]:
MILL_FITNESS_ADVANTAGE: float
MILL_REPLICATE: int


In [4]:
# Parameters
MILL_FITNESS_ADVANTAGE = 0.1
MILL_REPLICATE = 0


In [5]:
import copy
import random

import alifedata_phyloinformatics_convert as apc
from deap import algorithms as deap_algorithms
from deap import base as deap_base
from deap import creator as deap_creator
from deap import tools as deap_tools
from hstrat import hstrat
from keyname import keyname as kn
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp
from tqdm import tqdm

import pylib


In [6]:
random.seed(MILL_REPLICATE)


# Parameters


In [7]:
HSTRAT_RESOLUTION = 1000
HSTRAT_DIFFERENTIA_WIDTH = 64

TOURNAMENT_SIZE = 2

NGEN = 200

POPULATION_SIZE = 400


# Setup


In [8]:
deap_creator.create("FitnessMax", deap_base.Fitness, weights=(1.0,))
deap_creator.create("Individual", list, fitness=deap_creator.FitnessMax)

toolbox = deap_base.Toolbox()

toolbox.register(
    "individual",
    lambda *args, **kwargs: deap_creator.Individual([0.0]),
    deap_creator.Individual,
    n=1,
)
toolbox.register("population", deap_tools.initRepeat, list, toolbox.individual)


def evalOneMax(individual):
    return (individual[0] + random.random(),)


toolbox.register("evaluate", evalOneMax)
# hack to ensure gene annotation remains associated with gene
toolbox.register("mate", lambda x, y: random.choice([(x, y), (y, x)]))
toolbox.register("mutate", lambda x: tuple(x))
toolbox.register("select", deap_tools.selTournament, tournsize=TOURNAMENT_SIZE)

# Decorate the variation operators
toolbox.decorate("mate", pylib.deap.hstrat_mate_decorator)


# Evolve


In [9]:
# Create the population and populate the history
population = toolbox.population(n=POPULATION_SIZE)

species_ancestor = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=hstrat.nominal_resolution_algo.Policy(),
    stratum_differentia_bit_width=HSTRAT_DIFFERENTIA_WIDTH,
)
for member in population:
    member.species_annotation = species_ancestor.CloneDescendant()
    member.gene_annotation = hstrat.HereditaryStratigraphicColumn(
        stratum_retention_policy=hstrat.perfect_resolution_algo.Policy(),
        stratum_differentia_bit_width=HSTRAT_DIFFERENTIA_WIDTH,
        initial_stratum_annotation=0,
    ).CloneDescendant(0)


# Do the evolution, the decorators will take care of updating the
# history
prevalences = []
for gen in tqdm(range(NGEN)):
    prevalences.append(sum(1 for x in population if x[0] > 0))

    offspring = deap_algorithms.varAnd(
        population, toolbox, cxpb=1.0, mutpb=0.0
    )
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
    population = [copy.deepcopy(ind) for ind in population]

    if gen >= NGEN // 4:
        if prevalences[-1] == 0:
            population[0][0] = MILL_FITNESS_ADVANTAGE



  0%|                                                                                                                                                                 | 0/200 [00:00<?, ?it/s]

  0%|▊                                                                                                                                                        | 1/200 [00:02<06:52,  2.07s/it]

  1%|█▌                                                                                                                                                       | 2/200 [00:03<06:04,  1.84s/it]

  2%|██▎                                                                                                                                                      | 3/200 [00:05<06:19,  1.93s/it]

  2%|███                                                                                                                                                      | 4/200 [00:07<05:42,  1.75s/it]

  2%|███▊                                                                                                                                                     | 5/200 [00:09<05:45,  1.77s/it]

  3%|████▌                                                                                                                                                    | 6/200 [00:11<06:05,  1.89s/it]

  4%|█████▎                                                                                                                                                   | 7/200 [00:12<05:37,  1.75s/it]

  4%|██████                                                                                                                                                   | 8/200 [00:14<05:18,  1.66s/it]

  4%|██████▉                                                                                                                                                  | 9/200 [00:15<05:16,  1.66s/it]

  5%|███████▌                                                                                                                                                | 10/200 [00:17<05:11,  1.64s/it]

  6%|████████▎                                                                                                                                               | 11/200 [00:18<04:48,  1.53s/it]

  6%|█████████                                                                                                                                               | 12/200 [00:20<04:53,  1.56s/it]

  6%|█████████▉                                                                                                                                              | 13/200 [00:22<05:13,  1.68s/it]

  7%|██████████▋                                                                                                                                             | 14/200 [00:23<05:13,  1.69s/it]

  8%|███████████▍                                                                                                                                            | 15/200 [00:25<05:13,  1.69s/it]

  8%|████████████▏                                                                                                                                           | 16/200 [00:27<05:32,  1.81s/it]

  8%|████████████▉                                                                                                                                           | 17/200 [00:29<05:24,  1.78s/it]

  9%|█████████████▋                                                                                                                                          | 18/200 [00:31<05:45,  1.90s/it]

 10%|██████████████▍                                                                                                                                         | 19/200 [00:34<06:39,  2.21s/it]

 10%|███████████████▏                                                                                                                                        | 20/200 [00:36<06:30,  2.17s/it]

 10%|███████████████▉                                                                                                                                        | 21/200 [00:39<07:15,  2.43s/it]

 11%|████████████████▋                                                                                                                                       | 22/200 [00:42<07:10,  2.42s/it]

 12%|█████████████████▍                                                                                                                                      | 23/200 [00:45<07:48,  2.65s/it]

 12%|██████████████████▏                                                                                                                                     | 24/200 [00:47<07:28,  2.55s/it]

 12%|███████████████████                                                                                                                                     | 25/200 [00:49<07:16,  2.49s/it]

 13%|███████████████████▊                                                                                                                                    | 26/200 [00:52<06:54,  2.38s/it]

 14%|████████████████████▌                                                                                                                                   | 27/200 [00:55<07:29,  2.60s/it]

 14%|█████████████████████▎                                                                                                                                  | 28/200 [00:57<07:09,  2.50s/it]

 14%|██████████████████████                                                                                                                                  | 29/200 [01:00<08:02,  2.82s/it]

 15%|██████████████████████▊                                                                                                                                 | 30/200 [01:04<08:44,  3.08s/it]

 16%|███████████████████████▌                                                                                                                                | 31/200 [01:07<08:53,  3.16s/it]

 16%|████████████████████████▎                                                                                                                               | 32/200 [01:10<08:20,  2.98s/it]

 16%|█████████████████████████                                                                                                                               | 33/200 [01:13<07:54,  2.84s/it]

 17%|█████████████████████████▊                                                                                                                              | 34/200 [01:16<08:02,  2.91s/it]

 18%|██████████████████████████▌                                                                                                                             | 35/200 [01:19<08:13,  2.99s/it]

 18%|███████████████████████████▎                                                                                                                            | 36/200 [01:22<08:17,  3.03s/it]

 18%|████████████████████████████                                                                                                                            | 37/200 [01:26<08:54,  3.28s/it]

 19%|████████████████████████████▉                                                                                                                           | 38/200 [01:29<09:00,  3.34s/it]

 20%|█████████████████████████████▋                                                                                                                          | 39/200 [01:32<08:10,  3.05s/it]

 20%|██████████████████████████████▍                                                                                                                         | 40/200 [01:35<08:00,  3.00s/it]

 20%|███████████████████████████████▏                                                                                                                        | 41/200 [01:38<08:15,  3.12s/it]

 21%|███████████████████████████████▉                                                                                                                        | 42/200 [01:42<08:37,  3.28s/it]

 22%|████████████████████████████████▋                                                                                                                       | 43/200 [01:45<08:21,  3.20s/it]

 22%|█████████████████████████████████▍                                                                                                                      | 44/200 [01:48<08:15,  3.18s/it]

 22%|██████████████████████████████████▏                                                                                                                     | 45/200 [01:51<08:27,  3.28s/it]

 23%|██████████████████████████████████▉                                                                                                                     | 46/200 [01:54<07:57,  3.10s/it]

 24%|███████████████████████████████████▋                                                                                                                    | 47/200 [01:57<08:06,  3.18s/it]

 24%|████████████████████████████████████▍                                                                                                                   | 48/200 [02:01<08:23,  3.31s/it]

 24%|█████████████████████████████████████▏                                                                                                                  | 49/200 [02:06<09:58,  3.96s/it]

 25%|██████████████████████████████████████                                                                                                                  | 50/200 [02:10<09:53,  3.96s/it]

 26%|██████████████████████████████████████▊                                                                                                                 | 51/200 [02:15<10:22,  4.18s/it]

 26%|███████████████████████████████████████▌                                                                                                                | 52/200 [02:19<10:01,  4.07s/it]

 26%|████████████████████████████████████████▎                                                                                                               | 53/200 [02:23<09:50,  4.02s/it]

 27%|█████████████████████████████████████████                                                                                                               | 54/200 [02:26<09:33,  3.93s/it]

 28%|█████████████████████████████████████████▊                                                                                                              | 55/200 [02:30<09:01,  3.73s/it]

 28%|██████████████████████████████████████████▌                                                                                                             | 56/200 [02:33<08:43,  3.63s/it]

 28%|███████████████████████████████████████████▎                                                                                                            | 57/200 [02:38<09:12,  3.86s/it]

 29%|████████████████████████████████████████████                                                                                                            | 58/200 [02:42<09:28,  4.00s/it]

 30%|████████████████████████████████████████████▊                                                                                                           | 59/200 [02:45<08:49,  3.76s/it]

 30%|█████████████████████████████████████████████▌                                                                                                          | 60/200 [02:48<08:32,  3.66s/it]

 30%|██████████████████████████████████████████████▎                                                                                                         | 61/200 [02:54<09:26,  4.07s/it]

 31%|███████████████████████████████████████████████                                                                                                         | 62/200 [02:58<09:26,  4.10s/it]

 32%|███████████████████████████████████████████████▉                                                                                                        | 63/200 [03:01<08:55,  3.91s/it]

 32%|████████████████████████████████████████████████▋                                                                                                       | 64/200 [03:05<08:48,  3.88s/it]

 32%|█████████████████████████████████████████████████▍                                                                                                      | 65/200 [03:10<09:26,  4.20s/it]

 33%|██████████████████████████████████████████████████▏                                                                                                     | 66/200 [03:15<10:01,  4.49s/it]

 34%|██████████████████████████████████████████████████▉                                                                                                     | 67/200 [03:19<09:46,  4.41s/it]

 34%|███████████████████████████████████████████████████▋                                                                                                    | 68/200 [03:24<09:35,  4.36s/it]

 34%|████████████████████████████████████████████████████▍                                                                                                   | 69/200 [03:28<09:31,  4.36s/it]

 35%|█████████████████████████████████████████████████████▏                                                                                                  | 70/200 [03:32<09:12,  4.25s/it]

 36%|█████████████████████████████████████████████████████▉                                                                                                  | 71/200 [03:38<10:26,  4.85s/it]

 36%|██████████████████████████████████████████████████████▋                                                                                                 | 72/200 [03:42<09:53,  4.63s/it]

 36%|███████████████████████████████████████████████████████▍                                                                                                | 73/200 [03:47<10:05,  4.77s/it]

 37%|████████████████████████████████████████████████████████▏                                                                                               | 74/200 [03:53<10:50,  5.16s/it]

 38%|█████████████████████████████████████████████████████████                                                                                               | 75/200 [03:58<10:09,  4.88s/it]

 38%|█████████████████████████████████████████████████████████▊                                                                                              | 76/200 [04:03<10:20,  5.00s/it]

 38%|██████████████████████████████████████████████████████████▌                                                                                             | 77/200 [04:07<09:49,  4.79s/it]

 39%|███████████████████████████████████████████████████████████▎                                                                                            | 78/200 [04:12<09:35,  4.72s/it]

 40%|████████████████████████████████████████████████████████████                                                                                            | 79/200 [04:17<09:54,  4.91s/it]

 40%|████████████████████████████████████████████████████████████▊                                                                                           | 80/200 [04:23<10:15,  5.13s/it]

 40%|█████████████████████████████████████████████████████████████▌                                                                                          | 81/200 [04:29<10:39,  5.37s/it]

 41%|██████████████████████████████████████████████████████████████▎                                                                                         | 82/200 [04:34<10:24,  5.30s/it]

 42%|███████████████████████████████████████████████████████████████                                                                                         | 83/200 [04:39<09:58,  5.11s/it]

 42%|███████████████████████████████████████████████████████████████▊                                                                                        | 84/200 [04:43<09:39,  4.99s/it]

 42%|████████████████████████████████████████████████████████████████▌                                                                                       | 85/200 [04:48<09:28,  4.94s/it]

 43%|█████████████████████████████████████████████████████████████████▎                                                                                      | 86/200 [04:53<09:26,  4.97s/it]

 44%|██████████████████████████████████████████████████████████████████                                                                                      | 87/200 [04:59<09:46,  5.19s/it]

 44%|██████████████████████████████████████████████████████████████████▉                                                                                     | 88/200 [05:04<09:31,  5.10s/it]

 44%|███████████████████████████████████████████████████████████████████▋                                                                                    | 89/200 [05:09<09:18,  5.03s/it]

 45%|████████████████████████████████████████████████████████████████████▍                                                                                   | 90/200 [05:14<09:15,  5.05s/it]

 46%|█████████████████████████████████████████████████████████████████████▏                                                                                  | 91/200 [05:19<09:06,  5.02s/it]

 46%|█████████████████████████████████████████████████████████████████████▉                                                                                  | 92/200 [05:24<09:29,  5.27s/it]

 46%|██████████████████████████████████████████████████████████████████████▋                                                                                 | 93/200 [05:30<09:23,  5.26s/it]

 47%|███████████████████████████████████████████████████████████████████████▍                                                                                | 94/200 [05:36<09:37,  5.44s/it]

 48%|████████████████████████████████████████████████████████████████████████▏                                                                               | 95/200 [05:41<09:20,  5.33s/it]

 48%|████████████████████████████████████████████████████████████████████████▉                                                                               | 96/200 [05:45<08:53,  5.13s/it]

 48%|█████████████████████████████████████████████████████████████████████████▋                                                                              | 97/200 [05:51<09:18,  5.42s/it]

 49%|██████████████████████████████████████████████████████████████████████████▍                                                                             | 98/200 [05:56<08:55,  5.25s/it]

 50%|███████████████████████████████████████████████████████████████████████████▏                                                                            | 99/200 [06:01<08:48,  5.23s/it]

 50%|███████████████████████████████████████████████████████████████████████████▌                                                                           | 100/200 [06:07<08:46,  5.26s/it]

 50%|████████████████████████████████████████████████████████████████████████████▎                                                                          | 101/200 [06:12<08:29,  5.15s/it]

 51%|█████████████████████████████████████████████████████████████████████████████                                                                          | 102/200 [06:19<09:21,  5.73s/it]

 52%|█████████████████████████████████████████████████████████████████████████████▊                                                                         | 103/200 [06:24<08:48,  5.45s/it]

 52%|██████████████████████████████████████████████████████████████████████████████▌                                                                        | 104/200 [06:29<08:33,  5.35s/it]

 52%|███████████████████████████████████████████████████████████████████████████████▎                                                                       | 105/200 [06:35<08:53,  5.62s/it]

 53%|████████████████████████████████████████████████████████████████████████████████                                                                       | 106/200 [06:41<08:50,  5.65s/it]

 54%|████████████████████████████████████████████████████████████████████████████████▊                                                                      | 107/200 [06:46<08:40,  5.60s/it]

 54%|█████████████████████████████████████████████████████████████████████████████████▌                                                                     | 108/200 [06:52<08:34,  5.59s/it]

 55%|██████████████████████████████████████████████████████████████████████████████████▎                                                                    | 109/200 [06:59<09:06,  6.01s/it]

 55%|███████████████████████████████████████████████████████████████████████████████████                                                                    | 110/200 [07:04<08:48,  5.87s/it]

 56%|███████████████████████████████████████████████████████████████████████████████████▊                                                                   | 111/200 [07:10<08:37,  5.81s/it]

 56%|████████████████████████████████████████████████████████████████████████████████████▌                                                                  | 112/200 [07:16<08:28,  5.78s/it]

 56%|█████████████████████████████████████████████████████████████████████████████████████▎                                                                 | 113/200 [07:22<08:36,  5.94s/it]

 57%|██████████████████████████████████████████████████████████████████████████████████████                                                                 | 114/200 [07:28<08:40,  6.06s/it]

 57%|██████████████████████████████████████████████████████████████████████████████████████▊                                                                | 115/200 [07:33<08:12,  5.79s/it]

 58%|███████████████████████████████████████████████████████████████████████████████████████▌                                                               | 116/200 [07:39<07:55,  5.66s/it]

 58%|████████████████████████████████████████████████████████████████████████████████████████▎                                                              | 117/200 [07:45<08:13,  5.95s/it]

 59%|█████████████████████████████████████████████████████████████████████████████████████████                                                              | 118/200 [07:51<07:56,  5.81s/it]

 60%|█████████████████████████████████████████████████████████████████████████████████████████▊                                                             | 119/200 [07:57<08:03,  5.97s/it]

 60%|██████████████████████████████████████████████████████████████████████████████████████████▌                                                            | 120/200 [08:03<07:57,  5.97s/it]

 60%|███████████████████████████████████████████████████████████████████████████████████████████▎                                                           | 121/200 [08:08<07:31,  5.72s/it]

 61%|████████████████████████████████████████████████████████████████████████████████████████████                                                           | 122/200 [08:14<07:37,  5.86s/it]

 62%|████████████████████████████████████████████████████████████████████████████████████████████▊                                                          | 123/200 [08:20<07:18,  5.70s/it]

 62%|█████████████████████████████████████████████████████████████████████████████████████████████▌                                                         | 124/200 [08:27<07:51,  6.21s/it]

 62%|██████████████████████████████████████████████████████████████████████████████████████████████▍                                                        | 125/200 [08:33<07:43,  6.19s/it]

 63%|███████████████████████████████████████████████████████████████████████████████████████████████▏                                                       | 126/200 [08:39<07:29,  6.07s/it]

 64%|███████████████████████████████████████████████████████████████████████████████████████████████▉                                                       | 127/200 [08:46<07:47,  6.41s/it]

 64%|████████████████████████████████████████████████████████████████████████████████████████████████▋                                                      | 128/200 [08:52<07:31,  6.26s/it]

 64%|█████████████████████████████████████████████████████████████████████████████████████████████████▍                                                     | 129/200 [08:59<07:34,  6.40s/it]

 65%|██████████████████████████████████████████████████████████████████████████████████████████████████▏                                                    | 130/200 [09:05<07:26,  6.38s/it]

 66%|██████████████████████████████████████████████████████████████████████████████████████████████████▉                                                    | 131/200 [09:12<07:29,  6.51s/it]

 66%|███████████████████████████████████████████████████████████████████████████████████████████████████▋                                                   | 132/200 [09:20<07:43,  6.82s/it]

 66%|████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                  | 133/200 [09:26<07:35,  6.80s/it]

 67%|█████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                 | 134/200 [09:33<07:19,  6.65s/it]

 68%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                 | 135/200 [09:39<07:09,  6.61s/it]

 68%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                | 136/200 [09:46<06:56,  6.51s/it]

 68%|███████████████████████████████████████████████████████████████████████████████████████████████████████▍                                               | 137/200 [09:52<06:56,  6.61s/it]

 69%|████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                              | 138/200 [10:00<07:16,  7.04s/it]

 70%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                              | 139/200 [10:07<07:02,  6.92s/it]

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                             | 140/200 [10:15<07:06,  7.11s/it]

 70%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                            | 141/200 [10:21<06:52,  6.99s/it]

 71%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                           | 142/200 [10:29<06:56,  7.18s/it]

 72%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                           | 143/200 [10:37<07:05,  7.46s/it]

 72%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                          | 144/200 [10:43<06:39,  7.14s/it]

 72%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                         | 145/200 [10:51<06:34,  7.17s/it]

 73%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                        | 146/200 [10:57<06:08,  6.82s/it]

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                        | 147/200 [11:05<06:26,  7.29s/it]

 74%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                       | 148/200 [11:14<06:41,  7.72s/it]

 74%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                      | 149/200 [11:22<06:45,  7.95s/it]

 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                     | 150/200 [11:30<06:30,  7.81s/it]

 76%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                     | 151/200 [11:36<06:06,  7.48s/it]

 76%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                    | 152/200 [11:43<05:43,  7.15s/it]

 76%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                   | 153/200 [11:50<05:32,  7.07s/it]

 77%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                  | 154/200 [11:58<05:38,  7.37s/it]

 78%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                  | 155/200 [12:05<05:29,  7.33s/it]

 78%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                 | 156/200 [12:12<05:19,  7.26s/it]

 78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                | 157/200 [12:19<05:10,  7.22s/it]

 79%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                               | 158/200 [12:26<05:01,  7.18s/it]

 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                               | 159/200 [12:33<04:51,  7.10s/it]

 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                              | 160/200 [12:41<04:48,  7.22s/it]

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                             | 161/200 [12:48<04:36,  7.10s/it]

 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                            | 162/200 [12:54<04:26,  7.00s/it]

 82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                            | 163/200 [13:02<04:25,  7.18s/it]

 82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                           | 164/200 [13:10<04:23,  7.33s/it]

 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                          | 165/200 [13:18<04:28,  7.68s/it]

 83%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                         | 166/200 [13:27<04:34,  8.07s/it]

 84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                         | 167/200 [13:34<04:16,  7.77s/it]

 84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                        | 168/200 [13:42<04:04,  7.65s/it]

 84%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                       | 169/200 [13:48<03:49,  7.40s/it]

 85%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                      | 170/200 [13:56<03:48,  7.61s/it]

 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                      | 171/200 [14:03<03:32,  7.33s/it]

 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                     | 172/200 [14:11<03:30,  7.51s/it]

 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                    | 173/200 [14:19<03:29,  7.77s/it]

 87%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                   | 174/200 [14:27<03:20,  7.73s/it]

 88%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                  | 175/200 [14:34<03:07,  7.49s/it]

 88%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 176/200 [14:41<02:57,  7.39s/it]

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 177/200 [14:48<02:48,  7.35s/it]

 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                | 178/200 [14:56<02:42,  7.40s/it]

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               | 179/200 [15:04<02:41,  7.71s/it]

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉               | 180/200 [15:13<02:37,  7.88s/it]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋              | 181/200 [15:21<02:31,  7.99s/it]

 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍             | 182/200 [15:29<02:22,  7.94s/it]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 183/200 [15:38<02:19,  8.22s/it]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉            | 184/200 [15:46<02:11,  8.24s/it]

 92%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 185/200 [15:56<02:10,  8.69s/it]

 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍          | 186/200 [16:05<02:03,  8.79s/it]

 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏         | 187/200 [16:13<01:53,  8.74s/it]

 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉         | 188/200 [16:22<01:46,  8.85s/it]

 94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 189/200 [16:30<01:33,  8.50s/it]

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍       | 190/200 [16:39<01:27,  8.76s/it]

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 191/200 [16:47<01:16,  8.54s/it]

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉      | 192/200 [16:57<01:11,  8.89s/it]

 96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋     | 193/200 [17:07<01:03,  9.10s/it]

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍    | 194/200 [17:16<00:55,  9.23s/it]

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 195/200 [17:26<00:46,  9.28s/it]

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 196/200 [17:34<00:36,  9.00s/it]

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 197/200 [17:43<00:27,  9.12s/it]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 198/200 [17:52<00:17,  8.97s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 199/200 [18:01<00:08,  8.83s/it]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [18:12<00:00,  9.66s/it]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [18:12<00:00,  5.46s/it]

# Visualization


teeplots/./fitness-advantage=0.1+notebook=gene-selection-inference+replicate=0+viz=plot-sweep-and-annotations+ext=.pdf


teeplots/./fitness-advantage=0.1+notebook=gene-selection-inference+replicate=0+viz=plot-sweep-and-annotations+ext=.png


In [10]:
annotations = [
    s.GetAnnotation().bit_count()
    for s in population[0].gene_annotation.IterRetainedStrata()
]


def plot_sweep_and_annotations():
    # Create a figure and a single subplot
    fig, ax1 = plt.subplots()

    # Make the first plot on ax1
    sns.lineplot(data=annotations, color="b", ax=ax1)
    ax1.set_ylabel("Stratum Annotation Bit Count", color="b")
    ax1.tick_params("y", colors="b")

    # Create a second axes that shares the same x-axis
    ax2 = ax1.twinx()

    # Make the second plot on ax2
    sns.lineplot(data=prevalences, color="r", ax=ax2)
    ax2.set_ylabel("Gene Prevalence", color="r")
    ax2.tick_params("y", colors="r")

    ax1.set_xlabel("Generation")

    fig.tight_layout()  # otherwise the right y-label is slightly clipped


tp.tee(
    plot_sweep_and_annotations,
    teeplot_outattrs={
        "notebook": "gene-selection-inference",
        "replicate": str(MILL_REPLICATE),
        "fitness-advantage": str(MILL_FITNESS_ADVANTAGE),
    },
)


In [11]:
records = [
    {
        "notebook": "gene-selection-inference",
        "replicate": MILL_REPLICATE,
        "fitness-advantage": MILL_FITNESS_ADVANTAGE,
        "generation": generation,
        "prevalence": prevalence,
        "annotation": annotation,
    }
    for generation, (prevalence, annotation) in enumerate(
        zip(prevalences, annotations)
    )
]
outdf = pd.DataFrame.from_records(records)
outdf.to_csv(
    kn.pack(
        {
            "a": "prevalence-annotation-by-generation",
            "notebook": "gene-selection-inference",
            "replicate": str(MILL_REPLICATE),
            "fitness-advantage": str(MILL_FITNESS_ADVANTAGE),
            "ext": ".csv",
        }
    )
)
